In [3]:
import numpy as np

In [18]:
import xgboost as xgb

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
import category_encoders

In [11]:
features = pd.read_csv('features_08.csv')
kaggle = pd.read_csv('trocafone_kaggle_test.csv')
labels = pd.read_csv('labels_training_set.csv')

In [12]:
training = labels.set_index('person').join(features.set_index('person'))
train_labels = training['label']
train_data = training.drop('label',axis=1)

In [13]:
predict_data = kaggle.set_index('person').join(features.set_index('person'))

In [14]:
encoder = category_encoders.TargetEncoder(smoothing=True)
train_data_enc = encoder.fit_transform(np.array(train_data),np.array(train_labels))
predict_data_enc = encoder.transform(np.array(predict_data))

In [16]:
xg_classifier = XGBClassifier()

In [100]:
xtrain,xtest,ytrain,ytest = train_test_split(train_data_enc,np.array(train_labels))

In [101]:
train = xgb.DMatrix(data=xtrain,label=ytrain)
test = xgb.DMatrix(data=xtest,label=ytest)
data = xgb.DMatrix(data=train_data_enc,label=np.array(train_labels))
pred = xgb.DMatrix(data=predict_data_enc)
params = {'colsample_bytree': 0.9,
 'gamma': 2,
 'learning_rate': 0.016238752458245277,
 'max_depth': 13,
 'min_child_weight': 5,
 'n_estimators': 100,
 'subsample': 0.7847190225361189,
 'eval_metric':'auc'}

In [102]:
bst = xgb.train(params,train,200,[(train,'train'),(test,'eval')])

[22:08:16] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 98 extra nodes, 180 pruned nodes, max_depth=13
[0]	train-auc:0.861873	eval-auc:0.815167
[22:08:16] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 40 extra nodes, 324 pruned nodes, max_depth=7
[1]	train-auc:0.879115	eval-auc:0.841367
[22:08:16] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 76 extra nodes, 286 pruned nodes, max_depth=12
[2]	train-auc:0.891604	eval-auc:0.849247
[22:08:16] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 64 extra nodes, 250 pruned nodes, max_depth=13
[3]	train-auc:0.894521	eval-auc:0.852482
[22:08:16] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 270 pruned nodes, max_depth=9
[4]	train-auc:0.893289	eval-auc:0.851817
[22:08:16] C:\Users\Adminis

[22:08:20] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 354 pruned nodes, max_depth=13
[42]	train-auc:0.901278	eval-auc:0.860907
[22:08:20] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 34 extra nodes, 332 pruned nodes, max_depth=12
[43]	train-auc:0.901341	eval-auc:0.861117
[22:08:21] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 50 extra nodes, 252 pruned nodes, max_depth=13
[44]	train-auc:0.901702	eval-auc:0.860806
[22:08:21] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 36 extra nodes, 322 pruned nodes, max_depth=9
[45]	train-auc:0.903751	eval-auc:0.861401
[22:08:21] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 368 pruned nodes, max_depth=7
[46]	train-auc:0.903671	eval-auc:0.861266
[22:08:21] C:\Users\Ad

[22:08:25] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 40 extra nodes, 352 pruned nodes, max_depth=13
[85]	train-auc:0.907126	eval-auc:0.864269
[22:08:25] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 304 pruned nodes, max_depth=10
[86]	train-auc:0.907263	eval-auc:0.864063
[22:08:25] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 308 pruned nodes, max_depth=11
[87]	train-auc:0.90748	eval-auc:0.864007
[22:08:25] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 32 extra nodes, 374 pruned nodes, max_depth=13
[88]	train-auc:0.907481	eval-auc:0.863974
[22:08:25] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 386 pruned nodes, max_depth=11
[89]	train-auc:0.907454	eval-auc:0.864049
[22:08:26] C:\Users\A

[22:08:30] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 300 pruned nodes, max_depth=11
[127]	train-auc:0.912276	eval-auc:0.866679
[22:08:30] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 346 pruned nodes, max_depth=9
[128]	train-auc:0.912442	eval-auc:0.866659
[22:08:30] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 354 pruned nodes, max_depth=6
[129]	train-auc:0.91253	eval-auc:0.8667
[22:08:30] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 490 pruned nodes, max_depth=7
[130]	train-auc:0.912509	eval-auc:0.866635
[22:08:30] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 372 pruned nodes, max_depth=0
[131]	train-auc:0.912509	eval-auc:0.866635
[22:08:30] C:\Users\Adm

[22:08:34] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 374 pruned nodes, max_depth=12
[169]	train-auc:0.91733	eval-auc:0.867361
[22:08:34] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 374 pruned nodes, max_depth=0
[170]	train-auc:0.91733	eval-auc:0.867361
[22:08:34] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 336 pruned nodes, max_depth=0
[171]	train-auc:0.91733	eval-auc:0.867361
[22:08:34] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 368 pruned nodes, max_depth=0
[172]	train-auc:0.91733	eval-auc:0.867361
[22:08:34] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 368 pruned nodes, max_depth=0
[173]	train-auc:0.91733	eval-auc:0.867361
[22:08:35] C:\Users\Administ

In [80]:
bst = xgb.train(params,data,200)

[21:59:53] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 84 extra nodes, 330 pruned nodes, max_depth=13
[21:59:53] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 50 extra nodes, 364 pruned nodes, max_depth=12
[21:59:53] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 68 extra nodes, 440 pruned nodes, max_depth=10
[21:59:53] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 66 extra nodes, 344 pruned nodes, max_depth=12
[21:59:53] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 58 extra nodes, 378 pruned nodes, max_depth=12
[21:59:53] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 74 extra nodes, 304 pruned nodes, max_depth=11
[21:59:53] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:

[21:59:59] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 46 extra nodes, 516 pruned nodes, max_depth=12
[22:00:00] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 504 pruned nodes, max_depth=8
[22:00:00] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 356 pruned nodes, max_depth=6
[22:00:00] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 396 pruned nodes, max_depth=10
[22:00:00] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 46 extra nodes, 506 pruned nodes, max_depth=12
[22:00:00] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 48 extra nodes, 412 pruned nodes, max_depth=11
[22:00:00] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74

[22:00:06] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 8 extra nodes, 482 pruned nodes, max_depth=4
[22:00:06] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 332 pruned nodes, max_depth=8
[22:00:07] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 448 pruned nodes, max_depth=0
[22:00:07] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 532 pruned nodes, max_depth=8
[22:00:07] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 290 pruned nodes, max_depth=0
[22:00:07] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 240 pruned nodes, max_depth=9
[22:00:07] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree 

[22:00:12] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 32 extra nodes, 394 pruned nodes, max_depth=13
[22:00:12] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 448 pruned nodes, max_depth=11
[22:00:12] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 496 pruned nodes, max_depth=0
[22:00:13] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 318 pruned nodes, max_depth=8
[22:00:13] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 394 pruned nodes, max_depth=12
[22:00:13] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 564 pruned nodes, max_depth=0
[22:00:13] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: t

In [81]:
xg_prediction = bst.predict(pred)

In [82]:
submit = kaggle.set_index('person')

In [83]:
submit['label'] = xg_prediction

In [84]:
submit.to_csv('submit.csv')

In [97]:
bst.save_model('86605_xgboost_combinar_con_86602_catboost')

## Combinando con catboost (0.86605)

In [85]:
import catboost as cb

In [86]:
cb_classifier = cb.CatBoostClassifier(iterations=400,
                                     depth=8,
                                     l2_leaf_reg=100,
                                     random_strength=0.1,
                                     bagging_temperature=0,
                                     border_count=254,
                                     learning_rate=0.03)

In [87]:
cb_classifier.load_model('preentrenados/86602catboost')

In [88]:
cb_prediction = cb_classifier.predict_proba(predict_data)[:,1]

In [93]:
prediction = 0.9*cb_prediction+0.1*xg_prediction

In [94]:
submit = kaggle.set_index('person')

In [95]:
submit['label'] = prediction

In [96]:
submit.to_csv('submit.csv')